In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification, AdamWeightDecay
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np
import tensorflow as tf

# Load the TextPreprocessor class (assumed to be defined already)
from textpreprocessor import TextPreprocessor

# Initialize the Text Preprocessor
processor = TextPreprocessor()

# Load data
df_train, df_test = processor.load_data()

# Preprocess data
df_train = processor.preprocess(df_train)
df_test = processor.preprocess(df_test)

# Split data
X_train, y_train = processor.split_data(df_train)
X_test, y_test = processor.split_data(df_test)

/home/woong/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-19 23:42:13.982520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 23:42:13.991405: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 23:42:13.994029: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 23:42:14.001167: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimiz

Sampled Data Shape:
(10000, 3)
Sampled Data Shape:
(10000, 3)


In [2]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/home/woong/myenv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Tokenizing the datasets
X_train_tokens = tokenizer(
    text=list(X_train),
    add_special_tokens=True,
    max_length=100,
    padding='max_length',
    truncation=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True
)

I0000 00:00:1726782148.944528  120150 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726782148.956106  120150 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726782148.956165  120150 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726782148.960374  120150 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726782148.960422  120150 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [4]:
X_test_tokens = tokenizer(
    text=list(X_test),
    add_special_tokens=True,
    max_length=100,
    padding='max_length',
    truncation=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True
)

In [5]:
# Define BERT Model
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Use Hugging Face's AdamWeightDecay optimizer
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
# Compile the model using a standard loss function
loss_fn = SparseCategoricalCrossentropy(from_logits=True)
bert_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [7]:
# Model Summary
bert_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# # Train the model
# history = bert_model.fit(
#     {'input_ids': X_train_tokens['input_ids'], 'attention_mask': X_train_tokens['attention_mask']},
#     y_train,
#     validation_split=0.2,
#     epochs=3,
#     batch_size=16
# )

In [9]:
# Train the model
history = bert_model.fit(
    {'input_ids': X_train_tokens['input_ids'], 'attention_mask': X_train_tokens['attention_mask']},
    y_train,
    validation_data=({'input_ids': X_test_tokens['input_ids'], 'attention_mask': X_test_tokens['attention_mask']}, y_test),
    epochs=3,
    batch_size=16
)

Epoch 1/3
625/625 [==============================] - 135s 205ms/step - loss: 0.3117 - accuracy: 0.8710 - val_loss: 0.2237 - val_accuracy: 0.9130
Epoch 2/3
625/625 [==============================] - 114s 183ms/step - loss: 0.1465 - accuracy: 0.9493 - val_loss: 0.2381 - val_accuracy: 0.9114
Epoch 3/3
625/625 [==============================] - 112s 180ms/step - loss: 0.0733 - accuracy: 0.9771 - val_loss: 0.3277 - val_accuracy: 0.9041


In [10]:
# Predict on test data
y_pred_logits = bert_model.predict({'input_ids': X_test_tokens['input_ids'], 'attention_mask': X_test_tokens['attention_mask']}).logits
y_pred = np.argmax(y_pred_logits, axis=-1)

313/313 [==============================] - 26s 80ms/step


In [11]:
# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 90.41%
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.88      0.90      4972
           1       0.89      0.92      0.91      5028

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

